# Calculate incremental decomposition scheme for mechanosorptive strain  
### A. Ferrara, May 2025  

This notebook implements the incremental decomposition scheme proposed in [*Ferrara and Wittel (2025)*]() to isolate the mechanosorptive strain from the total strain measured with Digital Image Correlation (DIC). The analysis is applied to experimental results from **mechanosorptive creep tests** on **Norway spruce tissue slices** in the transverse directions, radial (R) and tangential (T), under cyclic relative humidity (RH) between 30% and 90%.  

Samples were cut in the orientations **{RL, RT, TR}**, where the first letter denotes the longitudinal dimension and the second the transverse width, all comprising alternating bands of earlywood (EW) and latewood (LW). Tests were carried out at **different loading degrees (LD)**: {RL, RT} at 30% and 50%, and {TR} at 40% of the respective tensile strength. Reference tensile strengths were taken from our earlier work ([Ferrara and Wittel, 2024](https://doi.org/10.1515/hf-2024-0046)).  

In the main folder, a subfolder must exist for each experiment (e.g., `MST30-9028`), containing:  
- `MST30-9028_Data_new.txt` with sample details.  
- A `Results` subfolder with one subfolder per sample, each containing a file `sample_name_strain_DIC.mat` generated at the end of the previous MATLAB analysis.  

In addition, the following files (retrievable from **_MS_Creep_Tests_Dataset_** ([10.17632/rsrsw8h7mv.1]())) must be available in the main folder:  
- **Tensile elastic tests**: mean elastic compliance values on Norway spruce tissue slices ([Ferrara and Wittel, 2024](https://doi.org/10.1515/hf-2024-0046)), enabling analyses of how creep compliance scales with elastic compliance.  
- **Tensile creep tests**: element compliances of the Kelvin–Voigt model describing mean viscoelastic compliances ([Ferrara and Wittel, 2025a](https://doi.org/10.1007/s11043-025-09772-1)), serving as input for calculating the viscoelastic strain component.  
- **Dynamic Vapor Sorption (DVS) tests**: an NPZ file with data from an RL-slice, following the RH profile applied in the mechanosorptive experiments.

#### *A. Ferrara and F.K. Wittel "Mechanosorptive Creep of Norway Spruce on the Tissue Scale Perpendicular to Grain"*, *Holzforschung* (2025)

### Imports and working path
Run the following section to import the required libraries and download the data folder in the current directory.

In [ ]:
# Imports
import os
import ast
from mat73 import loadmat
from math import *
import numpy as np
from scipy.signal import find_peaks
import pandas as pd
import glob
from datetime import datetime, timedelta
from collections import defaultdict
from scipy.optimize import lsq_linear
from scipy.optimize import least_squares
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import make_interp_spline
from scipy.stats import trim_mean
from sklearn.metrics import r2_score
from sklearn.isotonic import IsotonicRegression
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.lines import Line2D
import matplotlib.cm as cm

folder_path = '/home/aferrara/Desktop/Creep_experiments_1/'
# Set file paths
dvs_path = folder_path + 'dvs_data.npz'
elastic_path = folder_path + 'elastic_compliances_Ferrara_Wittel_2024.csv'
viscoel_path = folder_path + 'master_vec_prony_param.csv'

### Customized functions
Run the following section to make the custom functions available for use in the rest of the notebook.

In [ ]:
# Retrieve experiment path
def get_rawdata_path(exp_code, main_path):
    # Construct experiment path
    exp_path = os.path.join(main_path, exp_code)
    # Check if folder exists
    if not os.path.isdir(exp_path): raise FileNotFoundError(f"There is no folder for the experiment ({exp_path}).")
    # Check if the folder is empty
    if not os.listdir(exp_path): raise FileNotFoundError(f"The folder for the experiment exists but is empty ({exp_path}).")
    return exp_path
# end: get_rawdata_path

# Retrieve experiment data
def get_experiment_data(exp_code, main_path):

    # Get experiment folder path
    exp_path = get_rawdata_path(exp_code, main_path)
    data_path = os.path.join(exp_path, f"{exp_code}_Data_new.txt")

    # Check if file exists
    if not os.path.isfile(data_path): raise FileNotFoundError(f"Experiment data file '{data_path}' not found.")

    # Read the data file into a Pandas DataFrame
    df = pd.read_csv(data_path, delimiter='\t')  # adjust delimiter if needed

    # Convert DataFrame rows to a list of dictionaries
    samples_data = []
    for _, row in df.iterrows():
        sample = {
            "sample_holder": row[0],    # Column 1
            "exp_name": row[2],         # Column 3
            "img_name": row[3],         # Column 4
            "thick": row[10],           # Column 11
            "width": row[11],           # Column 12
            "area": row[10] * row[11],  # Column 11 * Column 12
            "load": row[6],             # Column 7
            "failure_load": row[7],     # Column 8
            "initial_load": row[8],     # Column 9
            "load_cell": row[9],        # Column 10
            "RH": row[12]               # Column 13
        }
        samples_data.append(sample)

    return samples_data, exp_path
# end: def get_experiment_data

# Retrieve all unique sample types across experiments
def get_sample_types(main_path, prefix="MST30-90"):
    folders = sorted(
        [d for d in os.listdir(main_path)
         if os.path.isdir(os.path.join(main_path, d)) and d.startswith(prefix)])
    types = set()
    for exp in folders:
        samples_data, _ = get_experiment_data(exp, main_path)
        for sample in samples_data:
            st = sample['exp_name'][3:5]
            if st == 'LT':
                tissue = "EW" if int(sample['exp_name'][-1]) >= 4 else "LW"
                st = f"LT-{tissue}"
            types.add(st)
    return sorted(types)
# end: def get_sample_types

# Format a list of strings for printing
def format_list(items):
    if not items:
        return ""
    if len(items) == 1:
        return items[0]
    if len(items) == 2:
        return " and ".join(items)
    return ", ".join(items[:-1]) + " and " + items[-1]
# end: def_format_llist

# Load strain results of DIC analysis
def load_strain_struct(grey_folder):
    # Find the first matching .mat file
    export_file_strain = glob.glob(os.path.join(grey_folder, '*_strain_DIC.mat'))[0]
    if not os.path.isfile(export_file_strain):
        raise FileNotFoundError(f"The data struct {os.path.basename(export_file_strain)} does not exist in {grey_folder}.")
    # Load mat file
    export_strain = loadmat(export_file_strain)['new_struct']
    # Convert MATLAB serial datenum to Python datetime 
    time_name = ['imgtime', 'loadtime', 'RHtime']
    for name in time_name:
        matlab_datenum = export_strain[name]  # MATLAB serial number
        if isinstance(matlab_datenum, np.ndarray):  # If it's an array, process each element
            export_strain[name] = np.array([
                (datetime.fromordinal(int(d)) + timedelta(days=d % 1) - timedelta(days=366)).replace(microsecond=0) for d in matlab_datenum])
        else:  # If it's a single value, process directly
            export_strain[name] = (datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum % 1) - timedelta(days=366)).replace(microsecond=0)

    return export_strain
# end: def load_strain_struct

# Locate previous and next index of drop 
def locate_drop_idx(cycle_idx, drop_idx, threshold=3):

    # Compute distances to each cycle boundary
    distances = np.abs(cycle_idx - drop_idx)
    min_dist = distances.min()

    if min_dist <= threshold:
        # If drop too close, snap to the nearest cycle boundary
        nearest = cycle_idx[np.argmin(distances)]
        drop_idx = nearest
        # Only look for the cycle boundary before drop
        before = cycle_idx[cycle_idx < drop_idx]
        before_idx = before.max() if before.size else None
        after_idx = drop_idx
    else:
        # If drop is not too close, find both before and after
        before = cycle_idx[cycle_idx < drop_idx]
        after  = cycle_idx[cycle_idx > drop_idx]
        before_idx = before.max() if before.size else None
        after_idx  = after.min() if after.size  else None

    return before_idx, drop_idx, after_idx
# end: locate_drop_idx

# Collect all samples matching a given sample type
def get_samples_by_type(sample_type, main_path, prefix="MST30-90"):
    
    matches = []
    folders = sorted(
        [d for d in os.listdir(main_path)
         if os.path.isdir(os.path.join(main_path, d)) and d.startswith(prefix)])

    for exp in folders:
        samples_data, exp_path = get_experiment_data(exp, main_path)
        for s in samples_data[::2]:
            st = s['exp_name'][3:5]
            if st == 'LT':
                tissue = "EW" if int(s['exp_name'][-1]) >= 4 else "LW"
                st = f"LT-{tissue}"
            if st == sample_type:
                matches.append((exp, exp_path, f"{s['exp_name']}_{s['sample_holder']}"))
                
    return matches
# end: get_samples_by_type

# Estract loading degree of a given sample
def extract_loading_deg(exp_path, sample_name):

    # Set path to image folder
    file_folder = os.path.join(exp_path, "Results", f"{sample_name}")
    # Check if folder exists
    if not os.path.isdir(file_folder): raise FileNotFoundError(f"The folder {file_folder} does not exist.")
    # Load data
    strain_results = load_strain_struct(file_folder)

    return strain_results['loading_deg']
# end: def extract_loading_deg

# Find indexes of local stress drop
def find_local_drop(arr, stress_drop=None, window=5):

    # Compute all differences
    arr = np.asarray(arr)
    n = arr.size
    if n < 2: raise ValueError("Array must have at least 2 elements to find a drop.")
    diffs = np.diff(arr)
    
    if stress_drop is None:
        # If no guess, then global search
        return int(np.argmin(diffs)) + 1

    # Clamp stress_drop to valid [0, n-1]
    sd = int(np.clip(stress_drop, 0, n - 1))
    # Convert to diffs‐index domain: diffs[j] = arr[j+1] - arr[j]
    # so arr‐index i corresponds to diffs‐index j = i-1
    j_center = sd - 1

    # Define window in diffs‐space
    j0 = max(0, j_center - window)
    j1 = min(n - 2, j_center + window)

    # Find drop
    local = diffs[j0:j1+1]
    if local.size:
        j_drop = j0 + int(np.argmin(local))
        return j_drop + 1

    # If for some reason the window was empty,
    # fall back to global search
    return int(np.argmin(diffs)) + 1
# end: def find_local_drop

# Build identical moisture cycles with corresponding time and stress
def build_cycles(tdata, wdata, total_cycles, stress_value, stressed_cycles):

    # Convert to arrays
    tdata = np.asarray(tdata)
    wdata = np.asarray(wdata)
    if tdata.shape != wdata.shape: raise ValueError("tdata and wdata must have the same shape")

    # Determine cycle period
    period = tdata[-1] - tdata[0] + tdata[-1] - tdata[-2]

    # Pre-allocate arrays
    n_per_cycle = tdata.size
    N = n_per_cycle * total_cycles
    t_full = np.empty(N, dtype=tdata.dtype)
    w_full = np.empty(N, dtype=wdata.dtype)
    s_full = np.empty(N, dtype=float)

    for cycle in range(total_cycles):
        start = cycle * n_per_cycle
        end = start + n_per_cycle

        # Time offset
        t_full[start:end] = tdata + cycle * period
        w_full[start:end] = wdata

        # Apply stress for initial cycles, else 0
        if cycle < stressed_cycles:
            s_full[start:end] = stress_value
        elif cycle == stressed_cycles:
            s_full[start-1] = 0.0
            s_full[start:end] = 0.0
        else:
            s_full[start:end] = 0.0

    return t_full, w_full, s_full
# end: build_cycles

# Build moisture cycles with different first sorption with corresponding time and stress
def build_cycles_with_initial(tdata0, wdata0, tdata, wdata, total_cycles, stress_value, stressed_cycles):
    
    # Convert to arrays
    t0 = np.asarray(tdata0)
    w0 = np.asarray(wdata0)
    t  = np.asarray(tdata)
    w  = np.asarray(wdata)

    # Determine cycle period
    dt = t[-1] - t[-2]
    period = (t[-1] - t[0]) + dt

    # Total length
    n0 = t0.size
    n  = t.size
    N  = n0 + n*(total_cycles-1)
    # Pre-allocate
    t_full = np.empty(N, dtype=t.dtype)
    w_full = np.empty(N, dtype=w.dtype)
    s_full = np.empty(N, dtype=float)

    # Set first cycle
    t_full[:n0] = t0
    w_full[:n0] = w0
    s_full[:n0] = stress_value if 0 < stressed_cycles else 0.0
    # Set remaining cycles
    for i in range(1, total_cycles):
        start = n0 + (i-1)*n
        end   = start + n
        # Time shift
        t_full[start:end] = t + (i-1)*period
        w_full[start:end] = w
        # Apply stress for initial cycles, else 0
        if i < stressed_cycles:
            s_full[start:end] = stress_value
        elif i == stressed_cycles:
            s_full[start-1] = 0.0
            s_full[start:end] = 0.0
        else:
            s_full[start:end] = 0.0

    return t_full, w_full, s_full
# end: build_cycles_with_initial

# Build moisture cycles with different first sorption from dvs data with corresponding time and stress
def build_dynamic_cycles(tdata0, wdata0, tdata, wdata, t_exp, cycle_idx, idx_before, idx_drop, stress_val):
    
    # Inititalize arrays
    tdata = np.asarray(tdata)
    wdata = np.asarray(wdata)
    if tdata.shape != wdata.shape: raise ValueError("tdata and wdata must have the same shape")
    
    # Create loading cycles with first sorption different
    n_pre = np.searchsorted(cycle_idx, idx_before,  side="right") + 1 if len(stress_val) > 1 else np.searchsorted(cycle_idx, idx_before,  side="right")
    t_full, w_full, s_full = build_cycles_with_initial(tdata0, wdata0, tdata, wdata, n_pre, stress_val[0], n_pre)

    if t_full[-1] > t_exp[idx_drop]:
        idx_closest = np.abs(t_full - t_exp[idx_drop-2]).argmin()
        s_full = s_full[:idx_closest+1]
        t_full = t_full[:idx_closest+1]
        w_full = w_full[:idx_closest+1]

    return t_full, w_full, s_full
# end: build_dynamic_cycles

# Find end index of each moisture cycle
def find_cycle_ends(w, tol=1e-5, merge_gap=5):
    w = np.asarray(w)
    w_min = w.min()

    # Find all 'low plateau' indices
    low_idxs = np.where(w <= (w_min + tol))[0]
    if low_idxs.size == 0:
        return []

    # Cluster them by index proximity
    clusters = [[low_idxs[0]]]
    for idx in low_idxs[1:]:
        if idx - clusters[-1][-1] <= merge_gap:
            # Same plateau cluster
            clusters[-1].append(idx)
        else:
            # New plateau cluster
            clusters.append([idx])

    # Pick the largest index from each cluster
    cycle_end_indices = [max(cluster) for cluster in clusters]

    return cycle_end_indices
# end: def find_cycle_ends

# Convert RH into mean w (mean S/D of tissues from standard dvs test)
def RH_to_w_mean(x):
    w = 7.155e-11 * x**5 - 1.659e-08 * x**4 + 1.75e-06 * x**3 - 9.343e-05 * x**2 + 0.003795 * x + 0.002295
    return w
# end: RH_to_w_mean

# Calculate hygroexpansion strain
def calculate_hygroexp_strain(sample_type, wdata):

    # Set hygroexpansion coefficients
    alpha = {"R": 0.182,
            "T": 0.343,
            "L": 0.0061}
    # Extract loading direction
    long_dir = sample_type[0]
    # Calculate hygroexpansion strain
    eps_w = alpha[long_dir]*(wdata-wdata[0])

    return eps_w
# end: def calculate_hygroexp_strain

# Fit elastic compliances to moisture
def fit_elastic_comp(elastic_path):

    # Read csv file
    df = pd.read_csv(elastic_path)
    # Group and compute mean compliance
    mean_df = df.rename(columns={'1/C0 [1/Mpa]': 'C0_mean'})
    # Calculate average w for each RH
    mean_df['w'] = RH_to_w_mean(mean_df['RH'])
    # Fit quadratic function in w
    el_fits = {}
    for stype, sub in mean_df.groupby('sample_type'):
        x = sub['w'].values
        y = sub['C0_mean'].values
        A = np.vstack(( x**2, x, np.ones_like(x) )).T
        # choose bounds per sample_type
        if stype in ("LR", "LT-LW"):
            lower = [-np.inf, 0.0,     0.0]
            upper = [ 0.0,     np.inf, np.inf]
        else:
            lower = [ 0.0,     0.0,     0.0]
            upper = [ np.inf, np.inf, np.inf]
        # Solve
        res = lsq_linear(A, y, bounds=(lower, upper))
        a, b, c = res.x
        el_fits[stype] = (a, b, c)

    return el_fits
# end: fit_elastic_comp

# Calculate elastic strain
def calculate_elastic_strain(comp_el, sample_type, stress, wdata, load=True):

    # Initialize array
    N = len(wdata)
    eps_el = [0.0] * N

    if not load:
        # All zeros if not loading
        return eps_el

    # Extract the three coefficients for compliance: a, b, c
    a, b, c = comp_el[sample_type]

    for i in range(N):
        # Compute compliance at wdata[i]
        C_i = a * wdata[i]**2 + b * wdata[i] + c
        # Instantaneous elastic strain = compliance * stress
        eps_el[i] = C_i * stress[i]

    return eps_el
# end: def calculate_elastic_strain

# Fit viscoelastic compliances (prony coeff.) to moisture
def fit_viscoelastic_comp(viscoel_path):

    # Read csv file
    df = pd.read_csv(viscoel_path, sep=",", dtype={"sample_type": str, "RH": float, "avg_comp_i": str})
    # Parse string‐list into float-list
    df["comp_i"] = df["avg_comp_i"].apply(lambda s: [float(x) for x in ast.literal_eval(s)])
    # Calculate average w for each RH
    df["w"] = RH_to_w_mean(df["RH"]) 
    # Fit quadratic function in w
    ve_fits = {}
    for stype, sub in df.groupby("sample_type"):
        w_vals = sub["w"].values
        C = np.vstack(sub["comp_i"].values)
        polys = [np.polyfit(w_vals, C[:, j], 2) for j in range(C.shape[1])]
        ve_fits[stype] = polys

    return ve_fits
# end: def fit_viscoelastic_comp

# Calculate prony series
def prony_response(comp_i, tdata, tau_0):
    return (np.sum(comp_i) - np.sum(comp_i[:, None] * np.exp(-tdata / tau_0[:, None]), axis=0))
# end: def prony_response

# Calculate prony coefficients at w
def comp_i_at_w(comp_ve, stype, wval):
    polys = comp_ve[stype]
    return np.array([ np.polyval(p, wval) for p in polys ])
# end: def comp_i_at_w

# Calculate viscoelastic creep compliance at given time and w
def compliance_at_t_w(comp_ve, stype, tdata, wval, tau_0):
    c_i = comp_i_at_w(comp_ve, stype, wval)
    return prony_response(c_i, tdata, tau_0)
# end: def compliance_at_t_w

# Calculate viscoelastic strain
def calculate_viscoelastic_strain(comp_ve, sample_type, stress, tdata, wdata, eps_el0):

    # Set retardation times [h]
    tau_0 = np.array([0.1, 1., 10., 100.])
    # Calculate viscoelastic strain
    eps_ve = []
    eps_i = np.zeros(tau_0.shape)
    eps_ve.append(eps_el0)
    for i in range(1, len(tdata)):
        # Calculate viscoelastic strain components
        comp_i_n1 = comp_i_at_w(comp_ve, sample_type, wdata[i])
        comp_i_n0 = comp_i_at_w(comp_ve, sample_type, wdata[i-1])
        comp_i = (comp_i_n1 + comp_i_n0) /2.
        deps_ve = np.zeros(tau_0.shape)
        deps_ve = 1. / tau_0 * (comp_i * stress[i] - eps_i)
        eps_i += deps_ve * (tdata[i]-tdata[i-1])
        eps_ve.append(np.sum(eps_i)+eps_el0)

    return eps_ve
# end: def calculate_viscoelastic_strain

# Calculate reference creep curves
def calculate_ref_viscoel_curves(comp_ve, sample_type, stress, tdata, wdata, eps_el0):

    # Set retardation times [h]
    tau_0 = np.array([0.1, 1., 10., 100.])
    # Calculate ref. curve at 30% RH = 0.07 mc
    eps_ve_ref30 = compliance_at_t_w(comp_ve, sample_type, tdata, min(wdata), tau_0) * stress + eps_el0
    # Calculate ref. curve at 90% RH = 0.20 mc
    eps_ve_ref90 = compliance_at_t_w(comp_ve, sample_type, tdata, max(wdata), tau_0) * stress + eps_el0

    return eps_ve_ref30, eps_ve_ref90
# end: def calculate_ref_viscoel_curves

# Fit monotonic regression
def unimodal_fit(x, y):

    # Initializations       
    x = np.asarray(x)
    y = np.asarray(y)
    N = len(y)
    best_err = np.inf
    best_fit = None

    # Try each possible peak-location k
    for k in range(1, N-1):
        # Rising isotonic on [0..k]
        ir_inc = IsotonicRegression(increasing=True)
        y_inc = ir_inc.fit_transform(x[:k+1], y[:k+1])

        # Falling  isotonic on [k..N-1]
        ir_dec = IsotonicRegression(increasing=False)
        y_dec = ir_dec.fit_transform(x[k:], y[k:])

        # Stitch (avoid doubling the k-th point)
        y_fit = np.concatenate([y_inc[:-1], y_dec])
        err = np.sum((y - y_fit)**2)
        if err < best_err:
            best_err  = err
            best_fit  = y_fit

    return best_fit
# end: def unimodal_fit

# Compute incremental strains across cycles
def compute_segment_deltas(tdata, edata, wdata, sdata, w_cycle_idx, n_segments=1000):

    # Initialize arrays
    seg_times = []
    seg_vals = []
    seg_stress = []
    seg_moist = []
    
    for i, end in enumerate(w_cycle_idx):
        # Set previous cycle
        start = -1 if i == 0 else w_cycle_idx[i-1]
        # Extract time
        t_c = tdata[start+1:end+1]
        rel = t_c - t_c[0]
        markers = np.linspace(0, rel[-1], n_segments)
        t_marks = markers + t_c[0]
        seg_times.append(t_marks)
        # Extract stress
        s_c = sdata[start+1:end+1]
        s_marks = UnivariateSpline(t_c, s_c, k=5, s=0)(t_marks)
        seg_stress.append(s_marks)
        # Extract moisture
        w_c = wdata[start+1:end+1]
        w_marks = UnivariateSpline(t_c, w_c, k=5, s=0)(t_marks)
        seg_moist.append(w_marks)
        # Extract strain
        e_c = edata[start+1:end+1]
        e_marks = UnivariateSpline(t_c, e_c, k=5, s=0)(t_marks)
        seg_vals.append(e_marks)

    # Convert to arrays
    seg_times = np.array(seg_times)
    seg_vals = np.array(seg_vals)
    seg_moist = np.array(seg_moist)
    seg_stress = np.array(seg_stress)

    # Calculate delta strain
    delta_s = seg_vals[1:] - seg_vals[:-1]

    return seg_times, delta_s, seg_moist, seg_stress
# end: compute_segment_deltas

# Overlap and fit cycles
def analyze_cycles(tdata, wdata, edata, w_cycle_idx, eps_cycle_idx, firstc=3, lastc=7, num_grid_points=1000):

    # Convert to arrays
    tdata = np.asarray(tdata)
    wdata = np.asarray(wdata)
    edata = np.asarray(edata)

    # Compute relative indices and segment boundaries
    firstc = firstc-2
    if lastc!=-1:
        lastc = lastc - 1
        w_cycle_idx = w_cycle_idx[firstc:lastc+1]
        eps_cycle_idx = eps_cycle_idx[firstc:lastc+1]
    else:
        w_cycle_idx = w_cycle_idx[firstc:]
        eps_cycle_idx = eps_cycle_idx[firstc:]
    w_segments = list(zip(w_cycle_idx, w_cycle_idx[1:]))
    eps_segments = list(zip(eps_cycle_idx, eps_cycle_idx[1:]))

    # Collect and shift cycles
    cycles = []
    for (w_start, w_end), (eps_start, eps_end) in zip(w_segments, eps_segments):
        # W segment and its relative time (0..1)
        t_w = tdata[w_start+1:w_end+1]
        w_seg = wdata[w_start+1:w_end+1]
        t_w_rel = (t_w - t_w[0]) / (t_w[-1] - t_w[0])
        # E segment and its relative time (0..1)
        t_e = tdata[eps_start+1:eps_end+1]
        e_seg = edata[eps_start+1:eps_end+1]
        t_e_rel = (t_e - t_e[0]) / (t_e[-1] - t_e[0])
        # Interpolate strain on the w grid
        e_on_w = np.interp(t_w_rel, t_e_rel, e_seg)
        # Locate peak
        peak_i = np.argmax(e_on_w)
        peak_val = e_on_w[peak_i]
        # Store cycle data
        cycles.append({
            't_seg_rel': t_w_rel,
            'w':       w_seg,
            'eps':       e_on_w,
            'last_eps':  e_on_w[-1],
            'peak_eps':  peak_val})

    ######## BUILD TEMPLATE FOR ALL CYCLES (except 1) ########
    # Use first cycle's peak as reference
    ref_peak = cycles[0]['peak_eps']
    # Apply vertical shift based on peaks
    for c in cycles: c['eps_shifted'] = c['eps'] - c['peak_eps'] + ref_peak
    # Build common time grid where all shifted cycles overlap
    t_fit = np.linspace(0, 1, num_grid_points)

    # Interpolate each shifted cycle onto the common grid
    s_interp = np.vstack([c['eps_shifted'] for c in cycles])
    avg_s_time = np.mean(s_interp, axis=0)
    avg_s_time = avg_s_time - avg_s_time[0]

    """
    plt.figure(figsize=(7,5))
    plt.plot(t_w_rel, avg_s_time, label="Mean", marker='o', color='blue')
    """

    s_interp = np.vstack([UnivariateSpline(c['t_seg_rel'], c['eps_shifted'], k=5, s=0.00001)(t_fit) for c in cycles])
    # Calculate average strain
    avg_s_time = np.mean(s_interp, axis=0)
    # Shift template to start from second cycle
    shift = avg_s_time[0] - edata[eps_cycle_idx[0]+1]
    avg_s_time = avg_s_time - shift

    # Try to fit avoiding any possible final increase
    if lastc != -1:       
        # Fit template
        y_fit = unimodal_fit(t_fit, avg_s_time)
        x = t_fit
        peak_idx = np.argmax(y_fit)
        # Fit a strictly‐decreasing isotonic to the tail
        ir_down = IsotonicRegression(increasing=False)
        x_tail = x[peak_idx:]
        y_tail = y_fit[peak_idx:]
        y_tail_iso = ir_down.fit_transform(x_tail, y_tail)
        # Stitch it back together
        avg_s_time = np.concatenate([y_fit[:peak_idx], y_tail_iso])


    ######## BUILD TEMPLATE FOR 1st CYCLE ########
    # Compute delta sorption
    w0 = wdata[:w_cycle_idx[0]+1]
    w1 = wdata[w_cycle_idx[0]+1:w_cycle_idx[1]+1]
    idx1 = w1.argmax()
    shift = w1[idx1] - w0[idx1]
    w1 = w1 - shift
    delta_w = [x/y for x, y in zip(w0, w1)]

    # Pick target times
    t_rel = (tdata[w_cycle_idx[0]+1:w_cycle_idx[1]+1] - tdata[w_cycle_idx[0]+1]) / (tdata[w_cycle_idx[1]+1] - tdata[w_cycle_idx[0]+1])
    t0_rel = t_rel[idx1]
    idx_fit = np.searchsorted(t_fit, t0_rel)
    slice_times = t_fit[:idx_fit+1]
    # Calculate scaling factor by interpolating delta_w
    scale = np.interp(slice_times, t_rel[:idx1+1], delta_w[:idx1+1])
    # Scale strain  proportional to sorption
    avg_s_time0 = np.concatenate((avg_s_time[:idx_fit+1] * scale, avg_s_time[idx_fit+1:]), axis=0)
    
    """
    plt.plot(t_fit, avg_s_time, label="Spline", color='red')
    plt.scatter(t_rel[idx1], avg_s_time[idx_fit], color='black', s = 50)
    plt.plot(t_fit, avg_s_time0, label="Scale", color='cyan')
    plt.xlabel("t [s]")
    plt.ylabel("Strain ε [-]")
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()
    """

    # Shift template I to start from elastic strain
    shift = avg_s_time0[0] - edata[0]
    avg_s_time0 = avg_s_time0 - shift

    # Shift template II to start from 0
    shift = avg_s_time[0]
    avg_s_time = avg_s_time - shift
    
    return [t_fit, avg_s_time0, avg_s_time]
# end: def analyze_cycles

# Calculate mechanosorptive strain from incremental scheme
def calculate_mechanosorptive_strain_from_inc(wdata, tdata, sdata, edata, w_cycle_idx, eps_cycle_idx, firstc_l=6, lastc_l=9, firstc_un=None, lastc_un=-1):
    
    #print(w_cycle_idx, len(w_cycle_idx), eps_cycle_idx, len(eps_cycle_idx))
    ########## Calculate strain increments ##########
    # Find strain templates
    [t_cycle, eps_cycle, eps_cycle1] = analyze_cycles(tdata, wdata, edata, w_cycle_idx, eps_cycle_idx)#, firstc=firstc_l, lastc=lastc_l)

    # Initialize arrays
    seg_times = []
    seg_vals = []
    seg_moist = []
    seg_stress = []
    
    for i, end in enumerate(w_cycle_idx):
        # Set previous cycle
        start = -1 if i == 0 else w_cycle_idx[i-1]
        # Extract time
        t_w = tdata[start+1:end+1]
        t_w_rel = (t_w - t_w[0]) / (t_w[-1] - t_w[0])
        # Extract moisture
        w_w = wdata[start+1:end+1]
        w_marks = UnivariateSpline(t_w_rel, w_w, k=5, s=0)(t_cycle)
        seg_moist.append(w_marks)
        # Extract stress
        s_w = sdata[start+1:end+1]
        s_marks = UnivariateSpline(t_w_rel, s_w, k=5, s=0)(t_cycle)
        seg_stress.append(s_marks)
    
    for i, end in enumerate(eps_cycle_idx):
        # Set previous cycle
        start = -1 if i == 0 else eps_cycle_idx[i-1]
        # Extract time
        t_eps = tdata[start+1:end+1]
        t_eps_rel = (t_eps - t_eps[0]) / (t_eps[-1] - t_eps[0])
        # Extract strain
        e_eps = edata[start+1:end+1]
        e_marks = UnivariateSpline(t_eps_rel, e_eps, k=5, s=0)(t_cycle)
        seg_vals.append(e_marks)

    ########## Correct jump of template I (1st cycle) ##########
    # Get corresponding moisture and time
    w_end = seg_moist[0][-1]
    w_temp = seg_moist[0][:len(seg_moist[0])//2]
    idx = np.abs(w_temp - w_end).argmin()
    t_start = t_cycle[idx]
    idx_t = np.abs(t_cycle - t_start).argmin()
    # Find delta strain
    delta_end = eps_cycle[-1] - eps_cycle[idx_t]
    x1, y1 = t_cycle[0], 0
    x2, y2 = t_cycle[-1], delta_end
    m0 = (y2 - y1) / (x2 - x1)
    b0 = y1 - m0 * x1
    # Shift jump by linear scaling
    shift_cycle = m0*t_cycle + b0 
    eps_cycle = eps_cycle - shift_cycle
    ########## Calculate mechanosorp. strain of 1st cycle ##########
    # Build temporary strain of first cycle
    shift = eps_cycle[0] - edata[0] # shift to start at initial elastic strain (if not done before)
    first_cycle = eps_cycle - shift
    first_cycle_ms = seg_vals[0] - first_cycle

    ########## Correct jump of template II (from 2nd cycle) ##########
    # Find delta strain
    delta_end = eps_cycle1[-1] - eps_cycle1[0]
    x1, y1 = t_cycle[0], eps_cycle1[0] - eps_cycle1[0]
    x2, y2 = t_cycle[-1], delta_end
    m1 = (y2 - y1) / (x2 - x1)
    b1 = y1 - m1 * x1
    # Shift jump by linear scaling
    shift_cycle = m1*t_cycle + b1
    eps_cycle1 = eps_cycle1 - shift_cycle

    ######### Calculate mechanosorp. strain of 1st cycle ##########
    second_cycle = eps_cycle1 - eps_cycle1[0] # shift to start at 0 (if not done before)
    seg_eps = []
    seg_eps.append(first_cycle_ms)
    for i in range(1,len(w_cycle_idx)):
        #print(len(seg_vals[i]), len(second_cycle), seg_vals[i][0])
        eps_inc = seg_vals[i] - (second_cycle + seg_vals[i][0]) + seg_eps[-1][-1]
        seg_eps.append(eps_inc)
    
    """plt.figure(figsize=(7,5))
    plt.plot(t_cycle, first_cycle_ms, label="mcs1")
    plt.plot(t_cycle, eps_cycle, label="Theta 1")
    plt.plot(t_cycle, seg_vals[0], label="eps_red")
    # Extract time
    t_eps = tdata[:eps_cycle_idx[0]+1]
    t_eps_rel = (t_eps - t_eps[0]) / (t_eps[-1] - t_eps[0])
    # Extract strain
    e_eps = edata[:eps_cycle_idx[0]+1]
    plt.plot(t_eps_rel, e_eps, label="eps_red_0")
    #plt.plot(t_cycle, eps_cycle1, label="Theta 2")
    plt.xlabel("t [s]")
    plt.ylabel("Strain ε [-]")
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()"""


    """if firstc_un is not None:
        ########## Find stress drop ##########
        # Find stress drop
        stress_flat = np.concatenate(seg_stress)
        drop_idx_flat = find_local_drop(stress_flat)
        # Fit unloaded cycles
        _, [t_end, eps_end] = analyze_cycles(tdata, wdata, edata, cycle_idx, firstc=firstc_un, lastc=lastc_un)
        ########## Correct jump of template III (last cycle) ##########
        delta_end = eps_end[-1] - eps_end[0]
        x1, y1 = t_cycle[0], eps_end[0] - eps_end[0]
        x2, y2 = t_cycle[-1], delta_end
        m = (y2 - y1) / (x2 - x1)
        b = y1 - m * x1
        shift_cycle = m*t_cycle + b 
        eps_end = eps_end - shift_cycle
        ########## Correct strain for unloading ##########
        # Build temporary strain of last cycle
        last_cycle = UnivariateSpline(t_end, eps_end, k=5, s=0)(seg_times[0])
        shift = last_cycle[0]
        last_cycle = last_cycle - shift

        M = seg_stress[0].shape[0]
        drop_cycle = drop_idx_flat // M
        inc_vals = []
        inc_vals.append(first_cycle_ms)
        for i,d in enumerate(cycle_idx):
            if i > 0:
                # Evaluate cycle at given segmented times
                idx = np.searchsorted(tdata, seg_times[i], side='left')
                idx = np.clip(idx, 0, len(tdata)-1)
                prev = np.clip(idx-1, 0, len(tdata)-1)
                choose_prev = np.abs(tdata[prev] - seg_times[0]) <= np.abs(tdata[idx] - seg_times[0])
                idx[choose_prev] = prev[choose_prev]
                epsP = edata[idx]
                
                if i > drop_cycle:
                    shift = last_cycle[0] - epsP[0]
                    inc_vals.append(epsP - last_cycle + shift + inc_vals[-1][-1])

                else:
                    shift = second_cycle[0] - epsP[0]
                    inc_vals.append(epsP - second_cycle + shift + inc_vals[-1][-1])

        # Flatten strain array        
        eps_flat  = np.concatenate(inc_vals)"""

    # Re-sample time array
    for i, end in enumerate(eps_cycle_idx):
        # Set previous cycle
        start = -1 if i == 0 else eps_cycle_idx[i-1]
        # Extract time
        t0 = tdata[start+1]
        t1 = tdata[end]
        seg_times.append(t_cycle * (t1 - t0) + t0)

    return first_cycle, second_cycle, second_cycle, seg_eps, seg_times, seg_moist, seg_stress
# end: def calculate_mechanosorptive_strain_from_inc

# Fitting model of mechanosorptive strain
def mechanosorptive_model(comp_j, stress, tdata, wdata):
    
    # Charachteristic moistures [-]
    mu_0 = np.array([1., 10., 100.])/100.
    # Build moisture‐rate array
    tdata = np.asarray(tdata)
    wdata = np.asarray(wdata)
    dt = np.diff(tdata)
    dw = np.diff(wdata)
    wrate = np.concatenate(([0.], np.abs(dw) / dt))
    
    # Initialize
    eps_j  = np.zeros_like(mu_0)
    eps_ms = np.zeros_like(wrate)

    # Calculate mechanosorptive strain
    for i in range(1, len(tdata)):
        delta_t = tdata[i] - tdata[i-1]
        # Calculate mechanosorptive increment
        deps = (wrate[i] / mu_0) * (comp_j * stress[i] - eps_j)
        eps_j  += deps * delta_t
        eps_ms[i] = eps_j.sum()
    
    return eps_ms
# end: def mechanosorptive_model

# Plot mechanosorptive strain vs moisture
def calculate_mechanosorptive_strain(sample_type, w_arr, t_arr, s_arr, eps_arr, w_cycle_idx, eps_cycle_idx):

    # Calculate strain components from total strain
    eps_comp = calculate_strain_components(sample_type, w_arr, t_arr, s_arr, eps_arr)
    ref_strain = eps_arr - eps_comp[5] # ref. strain = total - viscoel.

    # Calculate mechanosorptive strain from incremental scheme
    first_cycle, second_cycle , _, seg_eps, seg_times, seg_moist, seg_stress = calculate_mechanosorptive_strain_from_inc(eps_comp[1], eps_comp[0], eps_comp[2], ref_strain, w_cycle_idx, eps_cycle_idx)
    exp_eps_ms  = np.concatenate(seg_eps)
    times_flat  = np.concatenate(seg_times)
    w_flat = np.concatenate(seg_moist)
    s_flat = np.concatenate(seg_stress)

    # Fit incremental mechanosorptive strain
    initial_guess = np.array([0.01]*3, dtype=float) # initial guess
    # Residual function for least_squares
    def residual(comp_j):
        pred = mechanosorptive_model(comp_j, s_flat, times_flat, w_flat)
        return (pred - exp_eps_ms)
    # Bound compliances to be non‐negative
    result = least_squares(residual, initial_guess, bounds=(0, np.inf), xtol=1e-8, ftol=1e-8)
    comp_opt = result.x # get prony coefficients
    eps_ms_fit = mechanosorptive_model(comp_opt, s_flat, times_flat, w_flat) # fitted strain

    return times_flat, w_flat, s_flat, exp_eps_ms, eps_ms_fit, comp_opt
# end: def plot_mechanos_strain

# Calculate strain components (hygroexpansion, elastic, viscoelastic, mechanosorptive)
def calculate_strain_components(sample_type, wdata, tdata, sdata, edata):

    ########## Calculate hygroexpansion strain ##########
    hygroexp_strain = calculate_hygroexp_strain(sample_type, wdata)

    ########## Calculate elastic strain ##########
    comp_el = fit_elastic_comp(elastic_path)
    elastic_strain = calculate_elastic_strain(comp_el, sample_type, sdata, wdata)

    ########## Calculate viscoelastic strain ##########
    comp_ve = fit_viscoelastic_comp(viscoel_path)
    viscoel_strain = calculate_viscoelastic_strain(comp_ve, sample_type, sdata, tdata, wdata, 0)
    viscoel_ref30, viscoel_ref90 = calculate_ref_viscoel_curves(comp_ve, sample_type, sdata[0], tdata, wdata, 0)
    
    ########## Calculate mechanosorptive strain ##########
    combo_full = hygroexp_strain + elastic_strain + viscoel_strain
    mechanos_strain = edata - combo_full

    return [tdata, wdata, sdata, hygroexp_strain, elastic_strain, viscoel_strain, viscoel_ref30, viscoel_ref90, mechanos_strain, edata]
# end: def calculate_strains_component

# Build colormaps for loading degrees
def build_ld_colormaps(data, cmap_dict, scalar_range=(0.4, 0.9)):

    if cmap_dict is None: cmap_dict = {0.5: cm.Purples, 0.4: cm.Greens, 0.3: cm.Oranges}
    fallback_cmap = cm.Greys

    # Group datasets by LD
    grouped = defaultdict(list)
    for entry in data: grouped[entry['ld']].append(entry)

    # Generate list of colors for each group
    start, end = scalar_range
    colors_by_ld = {}
    rep_colors = {}

    # Sort by LD for deterministic behavior
    for ld_value in sorted(grouped.keys()):
        entries = grouped[ld_value]
        cmap = cmap_dict.get(ld_value, fallback_cmap)
        n = len(entries)

        if n > 1: scalars = np.linspace(start, end, n)
        else: scalars = [(start + end) / 2.0]

        cols = [cmap(s) for s in scalars]
        colors_by_ld[ld_value] = cols

        # Choose a representative color:
        # - third from the end if available, else the last available
        rep_idx = n - 3 if n >= 3 else n - 1
        rep_colors[ld_value] = cols[rep_idx]

    return colors_by_ld, rep_colors
# end: def build_ld_colormaps

# Set shared grid between primary and secondary y-axis
def set_shared_grid(ax1, ax2):

    # Grab the current primary ticks
    primary_ticks = ax1.get_yticks()
    # Keep the same gridlines at the primary_ticks
    ax1.set_yticks(primary_ticks)
    # Compute secondary ticks by slicing its data‐range into len(primary_ticks) points
    smin, smax = ax2.get_ylim()
    secondary_ticks = np.linspace(smin, smax, len(primary_ticks))
    ax2.set_yticks(secondary_ticks)
    ax2.set_yticklabels([str(round(t,3)) for t in secondary_ticks])

    return
# end: def set_shared_grid

# Customize plot
def customize_plot(ax1, ax2, fontsize):
    ax1.grid(True, alpha=0.5)
    # Customize axis
    ax1.set_xlabel('t [h]', fontsize=fontsize)
    ax1.set_xlim(left=0)
    ax1.set_ylabel(r'$\varepsilon$ [-]', fontsize=fontsize)
    ax1.set_ylim(bottom=0)
    ax1.tick_params(axis='both', labelsize=fontsize)
    ax2.tick_params(axis='both', labelsize=fontsize)
    return ax1, ax2
# end: customize_plot

# Set same number of thicks on twin axes
def match_tick_count(ax_src, ax_tgt, axis='y'):
    if axis not in ('x','y'):
        raise ValueError("`axis` must be 'x' or 'y'")

    # Pick off the existing tick *locations* on the source axis
    if axis == 'x':
        n = len(ax_src.get_xticks())-1
        locator = mticker.MaxNLocator(n)
        ax_tgt.xaxis.set_major_locator(locator)
    else:
        n = len(ax_src.get_yticks())-1
        locator = mticker.MaxNLocator(n)
        ax_tgt.yaxis.set_major_locator(locator)
    return
# end: def match_tick_count

# Set same tick values on twin axes
def match_tick_values(ax_src, ax_tgt, axis='y', copy_labels=True, copy_limits=True):
    if axis not in ('x','y'):
        raise ValueError("`axis` must be 'x' or 'y'")
    # 1) Optionally copy axis limits
    if copy_limits:
        if axis == 'y':
            ax_tgt.set_ylim(ax_src.get_ylim())
        else:
            ax_tgt.set_xlim(ax_src.get_xlim())

    # 2) Grab the *exact* tick positions
    if axis == 'y':
        ticks = ax_src.get_yticks()
        # use a FixedLocator so Matplotlib doesn't try to autoscale them away
        ax_tgt.yaxis.set_major_locator(mticker.FixedLocator(ticks))
        if copy_labels:
            labels = [t.get_text() for t in ax_src.get_yticklabels()]
            ax_tgt.set_yticklabels(labels)
    else:
        ticks = ax_src.get_xticks()
        ax_tgt.xaxis.set_major_locator(mticker.FixedLocator(ticks))
        if copy_labels:
            labels = [t.get_text() for t in ax_src.get_xticklabels()]
            ax_tgt.set_xticklabels(labels)
    return
# end: def match_tick_values

# Pick local strain minimum closest to end cycle
def snap_minima_to_cycles(min_indices, new_cycle_idx, *, max_gap=None, unique=True):
    
    min_indices = np.asarray(min_indices)
    anchors = np.asarray(new_cycle_idx)

    if min_indices.size == 0 or anchors.size == 0:
        return np.array([], dtype=int), np.array([], dtype=int), np.array([], dtype=int)

    # Work on a sorted copy of minima for fast nearest-neighbor via searchsorted
    mins_sorted = np.sort(np.unique(min_indices))
    pos = np.searchsorted(mins_sorted, anchors)

    left_idx = np.clip(pos - 1, 0, len(mins_sorted) - 1)
    right_idx = np.clip(pos,       0, len(mins_sorted) - 1)

    left_vals = mins_sorted[left_idx]
    right_vals = mins_sorted[right_idx]

    # Choose the closer side
    choose_right = np.abs(right_vals - anchors) < np.abs(left_vals - anchors)
    nearest = np.where(choose_right, right_vals, left_vals)
    dist = np.abs(nearest - anchors)

    # Apply max_gap filter if requested
    keep = np.ones(len(anchors), dtype=bool)
    if max_gap is not None:
        keep &= (dist <= max_gap)

    nearest = nearest[keep]
    anchors_kept = anchors[keep]
    dist = dist[keep]

    if not unique or nearest.size == 0:
        return nearest, anchors_kept, dist

    # Resolve duplicates: keep the anchor with smallest distance per chosen minimum
    # (ties resolved by first occurrence)
    order = np.lexsort((np.arange(len(nearest)), dist))  # sort by distance, then stable index
    nearest_sorted = nearest[order]
    anchors_sorted = anchors_kept[order]
    dist_sorted = dist[order]

    # Keep first occurrence of each chosen minimum
    uniq_mask = np.ones_like(nearest_sorted, dtype=bool)
    uniq_mask[1:] = nearest_sorted[1:] != nearest_sorted[:-1]

    picked_minima = nearest_sorted[uniq_mask]
    picked_anchors = anchors_sorted[uniq_mask]
    picked_dist = dist_sorted[uniq_mask]

    # (Optional) restore original anchor order
    sort_back = np.argsort(np.argsort(picked_anchors))
    return picked_minima[sort_back], picked_anchors[sort_back], picked_dist[sort_back]
# end: def snap_minima_to_cycles

# Detect unusual spikes in curve
def detect_single_point_spikes(y, k=5.0, eps=1e-12):
    y = np.asarray(y, float)
    dl = np.abs(y[1:-1] - y[:-2])
    dr = np.abs(y[1:-1] - y[2:])
    diff = np.diff(y)
    med = np.median(np.abs(diff - np.median(diff))) + eps
    scale = 1.4826 * med
    same_side = np.sign(y[1:-1] - y[:-2]) == np.sign(y[1:-1] - y[2:])
    spikes_mid = (dl > k*scale) & (dr > k*scale) & same_side
    idx = np.where(spikes_mid)[0] + 1
    return idx
# end: def detect_single_point_spikes

# Remove spikes by linear interpolation
def fix_spikes_linear(y, idx):
    y = np.asarray(y, float).copy()
    for i in idx:
        if 0 < i < len(y)-1:
            y[i] = 0.5*(y[i-1] + y[i+1])
    return y
# end: def fix_spikes_linear

### Load and prepare experimental datasets
Run the following section to load the raw data from the DVS test and retrieve all datasets corresponding to the available sample types **{RL, RT, TR}**.

For each sample type:  
- Experiments are grouped by loading degree (LD).  
- Raw strain data from DIC analysis are extracted, the pre-mechanosorption part is cut off, and the unloading phases are excluded as they yield unreliable results.
- Moisture cycles are reconstructed from the DVS test and aligned with stress and time, while strain data are up-sampled accordingly.  
- The elastic strain contribution is added based on the stress and the elastic compliances previously evaluated in [*Ferrara and Wittel (2024)*](https://doi.org/10.1515/hf-2024-0046).

In [ ]:
################## PREPARE EXPERIMENTAL DATASETS ##################
# Load data from dvs test
data = np.load(dvs_path)
wdata0 = data['w'][0] # moisture of 1st cycle
tdata0 = data['t'][0] # time of 1st cycle
wdata = data['w'][1] # moisture of following cycles
tdata = data['t'][1]  # time of following cycles

# Retrieve sample types
sample_types = get_sample_types(folder_path)[3:]
# Extract datasets
dataset = []
for sample_type in sample_types:
    
    # Find experiments of sample type
    matches = get_samples_by_type(sample_type, folder_path)
    
    # Group by loading degree
    grouped = defaultdict(list)
    for exp, exp_path, sample_name in matches:
        ld = np.round(extract_loading_deg(exp_path, sample_name), 1)
        grouped[ld].append((exp, exp_path, sample_name))
    
    # Build dataset entries for each loading degree and sample
    for ld, group in grouped.items():
        for idx, (exp, exp_path, sample_name) in enumerate(group):

            # Prepare file paths and load raw data
            file_folder = os.path.join(exp_path, "Results", sample_name)
            strain_results = load_strain_struct(file_folder)
            
            total_strain = strain_results['axial_strain']
            time = strain_results['exp_duration']
            stress = strain_results['stress_meas']
            
            # Cut off pre‑mechanosorption
            cycle_idx = find_cycle_ends(strain_results['RH_mean'], tol=1, merge_gap=5) # find cycle ends
            t0 = cycle_idx[0] # get first index = creep pre-mechanosrp.
            time = time[t0:] - time[t0]
            stress = stress[t0:]
            total_strain = total_strain[t0:] - total_strain[t0]
            cycle_idx = cycle_idx[1:] - t0
            
            # Determine load drop
            num_cycles = len(cycle_idx)
            if num_cycles > 8:
                # Drop unloading cycles
                guess_drop_idx = (cycle_idx[-2] + cycle_idx[-4]) / 2.
                drop_idx = find_local_drop(stress, stress_drop=guess_drop_idx, window=10)
                idx_before, drop_idx, idx_after = locate_drop_idx(cycle_idx, drop_idx)
                t_full, w_full, s_full = build_dynamic_cycles(tdata0, wdata0, tdata, wdata,
                    time, cycle_idx, idx_before, drop_idx, [stress[0], stress[-1]] )
            else:
                idx_before = drop_idx = idx_after = len(time) - 1
                t_full, w_full, s_full = build_dynamic_cycles(tdata0, wdata0, tdata, wdata,
                    time, cycle_idx, idx_before, drop_idx, [stress[0]])
            
            # Find new cycles index
            n_pre = np.searchsorted(cycle_idx, idx_before,  side="right")
            cycle_len = len(tdata)
            new_cycle_idx = [int((i+1)*cycle_len - 1) for i in range(n_pre)]
            if (sample_type == 'RT' and ld == 0.5 and idx == 4) or (sample_type == 'RT' and ld == 0.3 and idx in [1,2,3]):
                new_cycle_idx = np.concatenate([new_cycle_idx[:-1], [len(t_full)-1]])
            else:
                new_cycle_idx = np.concatenate([new_cycle_idx, [len(t_full)-1]])
            
            # Up-sample strain with a spline
            spline  = make_interp_spline(time, total_strain, k=5)
            eps_full = spline(t_full)
            # Find index of local minimum strains
            min_indices, _ = find_peaks(-eps_full)
            picked_minima, picked_anchors, distances = snap_minima_to_cycles(
                min_indices, new_cycle_idx, max_gap=None, unique=True)
            picked_minima = np.unique(picked_minima)
        
            # Cut last cycle out?
            t_sel = np.asarray(t_full)[new_cycle_idx]
            diffs = np.diff(t_sel)
            prev = diffs[:-1]
            last = diffs[-1]
            baseline = np.median(prev)
            tol = 0.5
            if abs(last - baseline) > tol:
                new_cycle_idx = new_cycle_idx[:-1]

            if len(picked_minima) < len(new_cycle_idx):
                picked_minima = np.append(picked_minima, new_cycle_idx[-1])
            picked_minima = np.sort(picked_minima)
            
            # Add initial elastic strain
            comp_el = fit_elastic_comp(elastic_path)
            eps_el = calculate_elastic_strain(comp_el, sample_type, s_full, w_full)
            eps_full = eps_full + eps_el[0]

            # Append to dataset, including sample_type
            dataset.append({
                'exp'         : exp,
                'sample_type' : sample_type,
                'sample_name' : sample_name,
                'ld'          : ld,
                't_full'      : t_full,
                'w_full'      : w_full,
                's_full'      : s_full,
                'eps_full'    : eps_full,
                'w_cycle_idx'   : new_cycle_idx,
                'eps_cycle_idx'   : picked_minima,
                'stress'      : stress})

print(f"Datasets for samples {format_list(sample_types)} have been retrieved.")

### Plot mechanosorptive strains
Run the following section to calculate and plot the mechanosorptive strains. The resulting strains are displayed grouped by sample type and LD, and the corresponding moisture cycles are plotted as well.

In [ ]:
################## CALCULATE AND PLOT MS STRAINS ##################
# Prepare figure
fontsize = 28
ylim1 = 0.04
xlim = 21.5
linewidth = 2
cmap_dict = {0.5: cm.Purples, 0.4: cm.Greens, 0.3: cm.Oranges}
_, rep_colors = build_ld_colormaps(dataset, cmap_dict) # create color map
fig, axes = plt.subplots(1, len(sample_types), figsize=(7 * len(sample_types), 6.5), constrained_layout=True)

# Loop over each sample type
results = []
for ax1, sample_type in zip(axes, sample_types):

    ax2 = ax1.twinx()
    # Get datasets for sample type
    entries = [e for e in dataset if e['sample_type'] == sample_type]
    # Group datasets by LD
    colors_by_ld, _ = build_ld_colormaps(entries, cmap_dict)

    # Loop through filtered datasets
    for entry in entries:
        # Extract arrays
        w_full = entry['w_full']
        t_full = entry['t_full']
        s_full = entry['s_full']
        eps_full = entry['eps_full']
        w_cycle_idx = entry['w_cycle_idx']
        eps_cycle_idx = entry['eps_cycle_idx']
        ld = entry['ld']
        sample_name = entry['sample_name']
        stress = entry['stress']
        # Set color
        color = colors_by_ld[ld].pop(0)      
        # Compute mechanosorptive strain
        #print(sample_name)
        t_flat, w_flat, s_flat, exp_eps_ms, eps_ms_fit, comp_opt = calculate_mechanosorptive_strain(sample_type, w_full, t_full, s_full, eps_full, w_cycle_idx, eps_cycle_idx)

        # Plot calculated strains
        ax1.plot(t_flat, exp_eps_ms, '-', color=color, linewidth=linewidth)
        
        # Store results
        results.append({
            'sample_name': sample_name,
            'sample_type': sample_type,
            'ld': ld,
            'time': t_flat,
            'stress': s_flat,
            'w': w_flat,
            'eps_ms_fit': eps_ms_fit,
            'eps_ms_exp': exp_eps_ms,
            'comp_opt': comp_opt,
            'compl_ms': eps_ms_fit/s_flat,
            'cycle_idx': cycle_idx,
            'stress_0': stress})
     
    # Plot moisture cycles 
    t_new, w_new, s_new = build_cycles_with_initial(tdata0, wdata0, tdata,  wdata, total_cycles=11, stress_value=s_flat[0], stressed_cycles=9)
    ax2.plot(t_new, w_new, linestyle='--', color='grey', label=r'$\omega$', linewidth = 1.5, alpha=0.5)
    # Customize axes
    ax1, ax2 = customize_plot(ax1, ax2, fontsize)
    ax1.locator_params(axis='x', nbins=5)
    ax1.set_xlim(right=xlim)
    ax1.set_ylim(top=ylim1)
    ax1.locator_params(axis='y', nbins=4)
    ax2.set_ylim(bottom = 0.07, top=0.2)
    
    set_shared_grid(ax1, ax2)
    ax2.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
    if sample_type == 'RT' or sample_type == 'TR':
        ax1.set_ylabel('')
        ax1.set_yticklabels([])
    else:
        ax1.set_ylabel(r'$\varepsilon^{ms}$ [-]', fontsize=fontsize)
    # Set reference axis for axes customization
    if sample_type == 'TR': ax2.set_ylabel(r'$\omega$ [-]', fontsize=fontsize, rotation=270, labelpad=32)
    if sample_type == 'RT' or sample_type == 'RL': ax2.set_yticklabels([])
    if sample_type == 'RL': ax_ref = ax1
    if sample_type == 'RT': ax_leg = ax1
    # Customize title
    ax1.set_title(sample_type, fontsize=fontsize)

# Customize legend
legend_handles = []
for ld, color in rep_colors.items():
    legend_handles.append(Line2D([0], [0], color=color, lw=linewidth, label=f'{int(ld*100)}% LD'))
legend_handles = sorted(legend_handles, key=lambda h: int(h.get_label().replace('% LD', '')))
legend_handles.append(Line2D([0], [0], color='black', lw=linewidth, linestyle='--', label='Mean fit'))
legend_handles.append(Line2D([0], [0], color='grey', lw=1.5, linestyle='--', alpha=0.5, label=r'$\omega$'))
ax_leg.set_zorder(ax2.get_zorder() + 1)
ax_leg.patch.set_alpha(0)
leg = ax_leg.legend(handles=legend_handles, fontsize=fontsize-4, loc='upper left')
leg.set_zorder=(1000)

# Show plot
plt.show()

### Plot KV-model fitting of mechanosorptive strains
Run the following section to calculate and plot the KV-model fitting of the mechanosorptive strains. The resulting strains are displayed grouped by sample type and LD.

In [ ]:
################## CALCULATE AND PLOT MS STRAINS FITTING ##################
# Prepare figure
ylim2 = ylim1
fig, axes = plt.subplots(1, len(sample_types), figsize=(6 * len(sample_types), 6), constrained_layout=True)

# Loop over each sample type
for ax1, sample_type in zip(axes, sample_types):

    ax2 = ax1.twinx()
    # Get datasets for sample type
    entries = [e for e in results if e['sample_type'] == sample_type]
    # Group datasets by LD
    colors_by_ld, _ = build_ld_colormaps(entries, cmap_dict)

    # Group by ld
    grouped = defaultdict(list)
    for r in entries: grouped[r['ld']].append(r)

    # Loop through filtered datasets
    for ld_val, group in grouped.items():
        eps_fit = []
        R2 = []
        for rec, color in zip(group, colors_by_ld[ld_val]):
            # Extract arrays
            t_flat = rec['time']
            eps_ms = rec['eps_ms_exp']
            w_flat = rec['w']
            s_flat = rec['stress']
            comp_j = rec['comp_opt']

            # Plot with relaxation
            t_new, w_new, s_new = build_cycles_with_initial(tdata0, wdata0, tdata,  wdata, total_cycles=11, stress_value=s_flat[0], stressed_cycles=9)
            eps_fit.append(mechanosorptive_model(comp_j, s_new, t_new, w_new))
            ax1.plot(t_new, eps_fit[-1], '-', color=color, linewidth=linewidth)
            
            # Calculate coefficient of determination
            eps_calc = mechanosorptive_model(comp_j, s_flat, t_flat, w_flat)
            R2.append(r2_score(eps_ms, eps_calc))
            
        # Calculate mean curve (exclude outliers)
        exp_eps_ms = trim_mean(eps_fit, 0.3, axis=0) #np.mean(eps_fit, axis=0)
        # Fit mean curve
        initial_guess = np.array([0.01]*3, dtype=float)
        # Residual function for least_squares
        def residual(comp_j):
            pred = mechanosorptive_model(comp_j, s_new, t_new, w_new)
            return (pred - exp_eps_ms)
        # Bound compliances to be non‐negative
        result = least_squares(residual, initial_guess, bounds=(0, np.inf), xtol=1e-12, ftol=1e-12)
        comp_mean = result.x # get prony coefficients
        eps_ms_mean = mechanosorptive_model(comp_mean, s_new, t_new, w_new) # mean fitted strain
        ax1.plot(t_new, eps_ms_mean, '--', color='black', linewidth=linewidth)

        # Calculate mean coefficient of determination
        R2_mean = np.mean(R2, axis=0)
        comp_str = np.array2string(comp_mean, precision=4, separator=", ")
        print(f"Sample: {sample_type:>6} | LD: {ld_val:6.2f} | "
            f"Cⱼ⁻¹: {comp_str} | R²: {R2_mean:.2f}")
        
    # Customize axes
    ax1, ax2 = customize_plot(ax1, ax2, fontsize)
    match_tick_values(ax_ref, ax1, axis='y', copy_labels=True, copy_limits=True)
    match_tick_values(ax_ref, ax1, axis='x', copy_labels=True, copy_limits=True)
    ax1.set_ylim(top=ylim2)
    ax1.set_xlim(right=xlim)
    ax2.set_yticks([])
    if sample_type == 'RT' or sample_type == 'TR':
        ax1.set_ylabel('')
        ax1.set_yticklabels([])
    else:
        ax1.set_ylabel(r'$\varepsilon^{ms}_{fit}$ [-]', fontsize=fontsize)
    if sample_type == 'RT': ax_leg = ax1
    # Customize title
    ax1.set_title(sample_type, fontsize=fontsize)
  
# Customize legend
legend_handles = []
for ld, color in rep_colors.items():
    legend_handles.append(Line2D([0], [0], color=color, lw=linewidth, label=f'{int(ld*100)}% LD'))
legend_handles = sorted(legend_handles, key=lambda h: int(h.get_label().replace('% LD', '')))
legend_handles.append(Line2D([0], [0], color='black', lw=linewidth, linestyle='--', label='Mean fit'))
ax_leg.set_zorder(ax2.get_zorder() + 1)
ax_leg.patch.set_alpha(0)
leg = ax_leg.legend(handles=legend_handles, fontsize=fontsize-4, loc='upper left')
leg.set_zorder=(1000)

# Show plot
plt.show()

### Plot mean fitted mechanosorptive strain rate

Run the following section to plot the evolution of mean fitted mechanosorptive strain rate grouped by sample type and LD.

In [ ]:
################## CALCULATE AND PLOT MS STRAIN RATE (TIME) ##################
# Prepare figure
fig, axes = plt.subplots(1, len(sample_types), figsize=(6 * len(sample_types), 6), constrained_layout=True)

# Loop over each sample type
for ax1, sample_type in zip(axes, sample_types):

    ax2 = ax1.twinx()
    # Get datasets for sample type
    entries = [e for e in results if e['sample_type'] == sample_type]
    # Group datasets by LD
    colors_by_ld, _ = build_ld_colormaps(entries, cmap_dict)

    # Group by ld
    grouped = defaultdict(list)
    for r in entries: grouped[r['ld']].append(r)

    # Loop through filtered datasets
    for ld_val, group in grouped.items():
        eps_fit = []
        s_store = []
        for rec, color in zip(group, colors_by_ld[ld_val]):
            # Extract arrays
            t_flat = rec['time']
            eps_ms_fit = rec['eps_ms_fit']
            w_flat = rec['w']
            s_flat = rec['stress']
            comp_j = rec['comp_opt']

            # Fit and extract compliance
            t_new, w_new, s_new = build_cycles_with_initial(tdata0, wdata0, tdata,  wdata, total_cycles=11, stress_value=s_flat[0], stressed_cycles=9)
            eps_fit.append(mechanosorptive_model(comp_j, s_new, t_new, w_new))
            
        # Calculate mean curve (exclude outliers)
        exp_eps_ms = trim_mean(eps_fit, 0.3, axis=0) #np.mean(eps_fit, axis=0)
        # Fit mean curve
        initial_guess = np.array([0.01]*3, dtype=float)
        # Residual function for least_squares
        def residual(comp_j):
            pred = mechanosorptive_model(comp_j, s_new, t_new, w_new)
            return (pred - exp_eps_ms)
        # Bound compliances to be non‐negative
        result = least_squares(residual, initial_guess, bounds=(0, np.inf), xtol=1e-12, ftol=1e-12)
        comp_mean = result.x # get prony coefficients
        eps_ms_mean = mechanosorptive_model(comp_mean, s_new, t_new, w_new) # mean fitted strain
        
        # Build strain‐rate array (time)
        dt = np.diff(t_new)
        dw = np.diff(w_new)
        deps = np.diff(eps_ms_mean)
        eps_t_rate = deps / dt
        t_filt = t_new[1:]
        # Filter out spikes
        idx = detect_single_point_spikes(eps_t_rate, k=5)
        eps_t_rate = fix_spikes_linear(eps_t_rate, idx)
        ax1.plot(t_filt, eps_t_rate, '-', color=rep_colors[ld_val], linewidth=linewidth)

    # Plot moisture
    ax2.plot(t_new, w_new, linestyle='--', color='grey', label=r'$\omega$', linewidth = 1.5, alpha=0.5)
    # Customize axes
    ax1, ax2 = customize_plot(ax1, ax2, fontsize)
    match_tick_values(ax_ref, ax1, axis='x', copy_labels=True, copy_limits=True)
    ax1.set_ylim(bottom= -0.006, top=0.012)
    match_tick_count(ax_ref, ax1, axis='y')
    ax1.set_xlim(right=xlim)
    ax2.set_ylim(bottom = 0.07, top=0.2)

    set_shared_grid(ax1, ax2)
    ax2.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
    if sample_type == 'TR': ax2.set_ylabel(r'$\omega$ [-]', fontsize=fontsize, rotation=270, labelpad=32)
    if sample_type == 'RT' or sample_type == 'RL': ax2.set_yticklabels([])
    if sample_type == 'RT' or sample_type == 'TR':
        ax1.set_ylabel('')
        ax1.set_yticklabels([])
    else:
        ax1.set_ylabel(r'$\Delta\overline{\varepsilon}^{{ms}}_{{fit}} / \Delta t$', fontsize=fontsize)
    if sample_type == 'RT': ax_leg = ax1
    # Customize title
    ax1.set_title(sample_type, fontsize=fontsize)

# Customize legend
legend_handles = []
for ld, color in rep_colors.items():
    legend_handles.append(Line2D([0], [0], color=color, lw=linewidth, label=f'{int(ld*100)}% LD'))
legend_handles = sorted(legend_handles, key=lambda h: int(h.get_label().replace('% LD', '')))
legend_handles.append(Line2D([0], [0], color='grey', lw=1.5, linestyle='--', alpha=0.5, label=r'$\omega$'))
ax_leg.set_zorder(ax2.get_zorder() + 1)
ax_leg.patch.set_alpha(0)
leg = ax_leg.legend(handles=legend_handles, fontsize=fontsize-4, loc='upper right')
leg.set_zorder=(1000)

# Show plot
plt.show()

### Plot mechanosorptive compliances
Run the following section to plot the mechanosorptive compliances calculated from the fitted mechanosorptive strains divided by stress. The resulting compliances are displayed grouped by sample type and LD.

In [ ]:
################## CALCULATE AND PLOT MS COMPLIANCES FITTING ##################
# Prepare figure
ylim3 = 24
fig, axes = plt.subplots(1, len(sample_types), figsize=(6 * len(sample_types), 6), constrained_layout=True)

# Loop over each sample type
for ax1, sample_type in zip(axes, sample_types):

    ax2 = ax1.twinx()
    # Get datasets for sample type
    entries = [e for e in results if e['sample_type'] == sample_type]
    # Group datasets by LD
    colors_by_ld, _ = build_ld_colormaps(entries, cmap_dict)

    # Group by ld
    grouped = defaultdict(list)
    for r in entries: grouped[r['ld']].append(r)

    # Loop through filtered datasets
    for ld_val, group in grouped.items():
        compl_fit = []
        eps_fit = []
        s_store = []
        for rec, color in zip(group, colors_by_ld[ld_val]):
            # Extract arrays
            t_flat = rec['time']
            eps_ms_fit = rec['eps_ms_fit']
            w_flat = rec['w']
            s_flat = rec['stress']
            comp_j = rec['comp_opt']

            # Fit and extract compliance
            t_new, w_new, s_new = build_cycles_with_initial(tdata0, wdata0, tdata,  wdata, total_cycles=9, stress_value=s_flat[0], stressed_cycles=9)
            eps_fit.append(mechanosorptive_model(comp_j, s_new, t_new, w_new))
            compl_fit.append(eps_fit[-1]/s_new) # [1/MPa]
            ax1.plot(t_new, compl_fit[-1]*1000, '-', color=color, linewidth=linewidth)
            s_store.append(s_new)
            
        # Calculate mean curve (exclude outliers)
        exp_eps_ms = trim_mean(eps_fit, 0.3, axis=0) #np.mean(eps_fit, axis=0)
        # Fit mean curve
        initial_guess = np.array([0.01]*3, dtype=float)
        # Residual function for least_squares
        def residual(comp_j):
            pred = mechanosorptive_model(comp_j, s_new, t_new, w_new)
            return (pred - exp_eps_ms)
        # Bound compliances to be non‐negative
        result = least_squares(residual, initial_guess, bounds=(0, np.inf), xtol=1e-12, ftol=1e-12)
        comp_mean = result.x # get prony coefficients
        eps_ms_mean = mechanosorptive_model(comp_mean, s_new, t_new, w_new) # mean fitted strain
        s_mean = np.mean(s_store) # mean stress [MPa]
        ax1.plot(t_new, eps_ms_mean/s_mean*1000, '--', color='black', linewidth=linewidth)

    # Customize axes
    ax1, ax2 = customize_plot(ax1, ax2, fontsize)
    match_tick_values(ax_ref, ax1, axis='x', copy_labels=True, copy_limits=True)
    ax1.set_ylim(top=ylim3)
    #ax1.locator_params(axis='y', nbins=5)
    match_tick_count(ax_ref, ax1, axis='y')
    ax1.set_xlim(right=xlim)
    ax2.set_yticks([])
    if sample_type == 'RT' or sample_type == 'TR':
        ax1.set_ylabel('')
        ax1.set_yticklabels([])
    else:
        ax1.set_ylabel(r'$M_c$ [1/GPa]', fontsize=fontsize)
    if sample_type == 'RT': ax_leg = ax1
    # Customize title
    ax1.set_title(sample_type, fontsize=fontsize)
  
# Customize legend
legend_handles = []
for ld, color in rep_colors.items():
    legend_handles.append(Line2D([0], [0], color=color, lw=linewidth, label=f'{int(ld*100)}% LD'))
legend_handles = sorted(legend_handles, key=lambda h: int(h.get_label().replace('% LD', '')))
legend_handles.append(Line2D([0], [0], color='black', lw=linewidth, linestyle='--', label='Mean fit'))
ax_leg.set_zorder(ax2.get_zorder() + 1)
ax_leg.patch.set_alpha(0)
leg = ax_leg.legend(handles=legend_handles, fontsize=fontsize-4, loc='upper left')
leg.set_zorder=(1000)

# Show plot
plt.show()

### Plot normalized mechanosorptive compliances
Run the following section to plot the normalized mechanosorptive compliances, obtained by dividing the mechanosorptive compliance by the elastic compliance at a given moisture content (set as input in the code). The resulting compliances are displayed grouped by sample type and LD.

In [ ]:
################## CALCULATE AND PLOT NORMALIZED MS COMPLIANCES ##################
# Extract elastic compliances
comp_el = fit_elastic_comp(elastic_path)
w_ref = 0.08 # set reference moisture content for normalized compliance

# Prepare figure
ylim4 = 9
fig, axes = plt.subplots(1, len(sample_types), figsize=(6 * len(sample_types), 6), constrained_layout=True)

# Loop over each sample type
for ax1, sample_type in zip(axes, sample_types):

    ax2 = ax1.twinx()
    # Get datasets for sample type
    entries = [e for e in results if e['sample_type'] == sample_type]
    # Group by ld
    grouped = defaultdict(list)
    for r in entries: grouped[r['ld']].append(r)

    # Loop through filtered datasets
    for ld_val, group in grouped.items():
        compl_fit = []
        eps_fit = []
        s_store = []
        for rec in group:
            # Extract arrays
            t_flat = rec['time']
            eps_ms_fit = rec['eps_ms_fit']
            w_flat = rec['w']
            s_flat = rec['stress']
            comp_j = rec['comp_opt']

            # Fit and extract compliance
            t_new, w_new, s_new = build_cycles_with_initial(tdata0, wdata0, tdata,  wdata, total_cycles=9, stress_value=s_flat[0], stressed_cycles=9)
            eps_fit.append(mechanosorptive_model(comp_j, s_new, t_new, w_new))
            compl_fit.append(eps_fit[-1]/s_new*1000) # [GPa]
            s_store.append(s_new)
            
        # Calculate mean curve (exclude outliers)
        exp_eps_ms = trim_mean(eps_fit, 0.3, axis=0) #np.mean(eps_fit, axis=0)
        # Fit mean curve
        initial_guess = np.array([0.01]*3, dtype=float)
        # Residual function for least_squares
        def residual(comp_j):
            pred = mechanosorptive_model(comp_j, s_new, t_new, w_new)
            return (pred - exp_eps_ms)
        # Bound compliances to be non‐negative
        result = least_squares(residual, initial_guess, bounds=(0, np.inf), xtol=1e-12, ftol=1e-12)
        comp_mean = result.x # get prony coefficients
        eps_ms_mean = mechanosorptive_model(comp_mean, s_new, t_new, w_new) # mean fitted strain
        s_mean = np.mean(s_store) # mean stress [MPa]
        compl_mean = eps_ms_mean / s_mean

        # Calculate elastic compliance
        a, b, c = comp_el[sample_type]
        C_i = a * w_ref**2 + b * w_ref + c
        # Plot normalized compliance
        ax1.plot(t_new, compl_mean/C_i, '-', color=rep_colors[ld_val], linewidth=linewidth)
    
    # Customize axes
    ax1, ax2 = customize_plot(ax1, ax2, fontsize)
    match_tick_values(ax_ref, ax1, axis='x', copy_labels=True, copy_limits=True)
    ax1.set_ylim(top=ylim4)
    match_tick_count(ax_ref, ax1, axis='y')
    ax1.set_xlim(right=xlim)
    ax2.set_yticks([])
    if sample_type == 'RT' or sample_type == 'TR':
        ax1.set_ylabel('')
        ax1.set_yticklabels([])
    else:
        ax1.set_ylabel(r'$\overline{M_c}/C_0^{-1}$ [-]', fontsize=fontsize)
    if sample_type == 'RT': ax_leg = ax1
    # Customize title
    ax1.set_title(sample_type, fontsize=fontsize)
  
# Customize legend
legend_handles = []
for ld, color in rep_colors.items():
    legend_handles.append(Line2D([0], [0], color=color, lw=linewidth, label=f'{int(ld*100)}% LD'))
legend_handles = sorted(legend_handles, key=lambda h: int(h.get_label().replace('% LD', '')))
ax_leg.set_zorder(ax2.get_zorder() + 1)
ax_leg.patch.set_alpha(0)
leg = ax_leg.legend(handles=legend_handles, fontsize=fontsize-4, loc='upper left')
leg.set_zorder=(1000)

# Show plot
plt.show()